#### concepts to know about
- partial
- __call__ for classes
- *args and **kwargs
- callbacks
- __dunder__ thingies

In [2]:
import torch
import matplotlib.pyplot as plt
import random

### Callbacks

In [4]:
import ipywidgets as widgets

In [5]:
w = widgets.Button(description='Click me')

In [7]:
def f(o): print('hi')

In [8]:
w.on_click(f)

In [9]:
w

Button(description='Click me', style=ButtonStyle())

In [12]:
w.click()

hi


In [13]:
from time import sleep

In [14]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [15]:
slow_calculation()

30

In [16]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb:
            cb(i)
    return res

In [22]:
def show_progress(epoch):
    print(f'Awesome! Epoch: {epoch}')

In [23]:
slow_calculation(show_progress)

Awesome! Epoch: 0
Awesome! Epoch: 1
Awesome! Epoch: 2
Awesome! Epoch: 3
Awesome! Epoch: 4


30

In [24]:
slow_calculation(lambda epoch: print(f'Awesome! Epoch: {epoch}'))

Awesome! Epoch: 0
Awesome! Epoch: 1
Awesome! Epoch: 2
Awesome! Epoch: 3
Awesome! Epoch: 4


30

In [25]:
def show_progress(exclamation, epoch):
    print(f'Awesome! {exclamation}, epoch: {epoch}')

In [28]:
slow_calculation(lambda epoch: show_progress('Job', epoch))

Awesome! Job, epoch: 0
Awesome! Job, epoch: 1
Awesome! Job, epoch: 2
Awesome! Job, epoch: 3
Awesome! Job, epoch: 4


30

In [29]:
def f(a, b, c):
    print(a, b, c)

In [30]:
from functools import partial

In [31]:
f(1, 2, 3)

1 2 3


In [38]:
g = partial(f, 1, 2)

In [39]:
g(3)

1 2 3


In [40]:
def make_show_progress(exclamation):
    def _inner(epoch):
        print(f'Awesome job {exclamation}, epoch: {epoch}')
    return _inner

In [41]:
slow_calculation(make_show_progress("Nice"))

Awesome job Nice, epoch: 0
Awesome job Nice, epoch: 1
Awesome job Nice, epoch: 2
Awesome job Nice, epoch: 3
Awesome job Nice, epoch: 4


30

In [42]:
slow_calculation(partial(show_progress, 'Nice'))

Awesome! Nice, epoch: 0
Awesome! Nice, epoch: 1
Awesome! Nice, epoch: 2
Awesome! Nice, epoch: 3
Awesome! Nice, epoch: 4


30

### Callbacks as callable classes

In [47]:
class ProgressShowingCallback():
    def __init__(self, exclamation): self.exclamation = exclamation
    def __call__(self, epoch):
        print(f'Awesome! {self.exclamation}, epoch: {epoch}')

In [48]:
cb = ProgressShowingCallback("Nice")

In [49]:
cb(1)

Awesome! Nice, epoch: 1


In [50]:
slow_calculation(cb)

Awesome! Nice, epoch: 0
Awesome! Nice, epoch: 1
Awesome! Nice, epoch: 2
Awesome! Nice, epoch: 3
Awesome! Nice, epoch: 4


30

### Multiple callback funcs; *args and **kwargs

In [55]:
def f(*args, **kwargs):
    # print(type(args), type(kwargs))
    print(f"args: {args}, kwargs: {kwargs}")

In [56]:
f(3, 'a', thing1='hello')

args: (3, 'a'), kwargs: {'thing1': 'hello'}


In [57]:
def g(a, b, c=0):
    print(a, b, c)

In [60]:
args=[1,2]
kwargs={'c': 3}
g(*args, **kwargs)

1 2 3


- Passing args and kwargs
- Functions which accept these as params
- Can be used to abosrb all the unused args as well

In [78]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb: cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb: cb.after_calc(i, res)
    
    return res

In [69]:
class PrintStepCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwarfs): print('Done')

In [66]:
slow_calculation(PrintStepCallback())

About to start
Done
About to start
Done
About to start
Done
About to start
Done
About to start
Done


30

In [81]:
class PrintStatusCallback():
    def before_calc(self, *args, **kwargs): print('About to start')
    def after_calc(self, *args, **kwargs): print(f"Epoch: {args[0]}, val: {args[1]}")

In [82]:
slow_calculation(PrintStatusCallback())

About to start
Epoch: 0, val: 0
About to start
Epoch: 1, val: 1
About to start
Epoch: 2, val: 5
About to start
Epoch: 3, val: 14
About to start
Epoch: 4, val: 30


30

In [83]:
class SlowCalculator():
    def __init__(self, cb=None):
        self.cb,self.res = cb, 0
        
    def callback(self, cb_name, *args):
        if not self.cb:
            return
        cb = getattr(self.cb, cb_name, None)
        if cb: return cb(self, *args)
    
    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i * i
            sleep(1)
            if self.callback('after_calc', i):
                print('early stop')
                break

In [90]:
class ModifyingCallback():
    def after_calc(self, calc, epoch):
        print(f'After {epoch}: {calc.res}')
        if calc.res > 10:
            return True
        if calc.res < 3:
            calc.res = calc.res * 2
        

In [97]:
c = SlowCalculator(ModifyingCallback())

In [92]:
c.calc()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
early stop


In [93]:
c.res

15

In [98]:
getattr(c, 'calc')()

After 0: 0
After 1: 1
After 2: 6
After 3: 15
early stop


In [99]:
class SloppyAdder():
    def __init__(self, o): self.o=o
    def __add__(self, s): return SloppyAdder(self.o + s.o + 0.01)
    def __repr__(self): return str(self.o)

In [100]:
s1 = SloppyAdder(1)
s2 = SloppyAdder(2)
s1+s2

3.01

In [117]:
class B:
    a,b=1,2
    def __getattr__(self, k):
        if k[0]=='_': raise AttributeError(k)
        return f'Hello {k}'
    
    def abcd(self):
        return 'abcd'

In [119]:
b = B()

In [103]:
b.a

1

In [104]:
b.foo

'Hello foo'

In [108]:
getattr(b, 'a')

1

In [129]:
getattr(b, 'abc')

'Hello abc'

In [120]:
b.abcd()

'abcd'

In [128]:
hasattr(b, 'abcd')

True